In [1]:
import openai
import json 
import pandas as pd
import time
from credentials import OPENAI_KEY
import os


openai.api_key = OPENAI_KEY
model_engine="text-davinci-003"

max_summaries = 10

in_filepath = "test_data/01_youtubeStatsTranscripts.json"

summ_filepath = "test_data/summaries"
if not os.path.exists(summ_filepath):
    os.makedirs(summ_filepath)
    
qa_filepath = "test_data/QA_out"
if not os.path.exists(qa_filepath):
    os.makedirs(qa_filepath)
    
out_filepath = "test_data/02_QAOutputChatGPT.json"

In [2]:
with open(in_filepath, 'r') as f:
    data = json.load(f)

In [3]:
data.keys()
len(data['videoId'])

100

In [5]:

####SUMMARIZE LONGER TRANSCRIPTS - store in summaries_dict
#### save checkpoints to summaries folder
all_summaries = []
summaries_dict = {
    
}
max_len = 1800
threshold_perc = 0.75
threshold = max_len*threshold_perc
chunksize = 1000
ind = 0 #used to iterate through transcript chunks to summarize

# summary_prefix = f"""This is a transcript from a YouTube video.in {threshold} words or less, 
#                     summarize the following text: """
# classify_prefix = "This is a transcript from a YouTube video."
for i,text in enumerate(data['transcript_strings']):
    video_id = data['videoId'][i]
    
#     if text and i <=250:
    if text and i <= max_summaries:
        tokens = text.split()
#         print(f"FINAL LENGTH: {len(tokens)}")
        if len(tokens) >= threshold_perc:
            summary_chunks = []
            for i in range(0, len(tokens),chunksize):
                ###SET indices to get tokens for each chunk
                if i == 0:
#                     print("---START")
                    start_ind = i
                    end_ind = chunksize
                elif len(tokens) % i < chunksize and len(tokens)//i <= 1.0:
#                     print("---END")
                    start_ind = i +1
                    end_ind = len(tokens)
                else:
#                     print("---MID")
                    start_ind = i +1
                    end_ind = i + chunksize
#                 print(f"START AND END: {start_ind} to {end_ind}")
#                 print("****")
                to_summarize = " ".join(tokens[start_ind:end_ind])
#                 print(to_summarize)
#                 print("****")

                summ_query = f"The following is a portion of a YouTube video transcript. Summarize what the speaker is saying and include as many details as you can about the speaker's current and former job positions. Also include details related to their reasons for quitting: {to_summarize}"
#                 print(len(summ_query.split()))
                summary_completion = openai.Completion.create(
                    engine=model_engine,
                    prompt=summ_query,
                    max_tokens=500, #prompt restricts length of response, but max_tokens determines cutoff tokens 
                    n=1,
                    stop=None,
                    temperature=0.5,
                )
#                 print(summary_completion)
                summary_chunks.append(summary_completion.choices[0].text)
            all_summaries.append("\n".join(summary_chunks))
            summaries_dict[video_id] = "\n".join(summary_chunks)
#             print(len("\n".join(summary_chunks).split()))
        else:
            print("CLASSIFY")
            continue
        print("---")
    else:
#         print("NO TRANSCRIPT")
        summaries_dict[video_id] = "NO TRANSCRIPT AVAILABLE"
    
    ###ADD CHECKPOINT IN 'summaries' file - put last video id into file name
    if len(str(ind)) ==1:
        checkpoint_ind = "00" + str(ind)
    elif len(str(ind)) == 2:
        checkpoint_ind = "0" + str(ind)
    else:
        checkpoint_ind = str(ind)
    checkpoint_file = f"{summ_filepath}/checkpoint_{checkpoint_ind}_{video_id}.json"
    with open(checkpoint_file, 'w', encoding='utf-8') as f:
        json.dump(summaries_dict, f, ensure_ascii=False, indent=4)
    
    if ind == max_summaries:
        break
    ind += 1

---
---
---
---
---
---
---
---
---


In [77]:
# text = summaries.values()
# qa_query = f"This is a transcript from a YouTube video. in three separate bullet points, answer 3 questions for the following transcript: \n What is the current job/occupation of the speaker? \n 2. What was the former job/occupation of the speaker? \n 3. What is the main reason that the speaker had for quitting their job/occupation? \n {text}"
    
# qa_completion = openai.Completion.create(
#     engine=model_engine,
#     prompt=qa_query,
#     max_tokens=500, #prompt restricts length of response, but max_tokens determines cutoff tokens 
#     n=1,
#     stop=None,
#     temperature=0.5,
# )

# qa_result = qa_completion.choices[0].text
# qa_result

In [8]:
#answer questions
#get last item from chkpt_filpath - last checkpoint w most summaries 
final_summaries = sorted(os.listdir(summ_filepath))[-1]
summaries_filepath=f"{summ_filepath}/{final_summaries}"
with open(summaries_filepath, 'r') as f:
    summaries = json.load(f)

qa_dict = {}
qa_ind = 0 
for video_id, text in summaries.items():
    qa_query = f"This is a transcript from a YouTube video. in three separate bullet points, answer 3 questions for the following transcript: \n What is the current job/occupation of the speaker? \n 2. What was the former job/occupation of the speaker? \n 3. What is the main reason that the speaker had for quitting their job/occupation? \n {text}"
    if len(qa_query.split()) <= 3000:
        qa_completion = openai.Completion.create(
            engine=model_engine,
            prompt=qa_query,
            max_tokens=500, #prompt restricts length of response, but max_tokens determines cutoff tokens 
            n=1,
            stop=None,
            temperature=0.5,
        )

        qa_result = qa_completion.choices[0].text
#         print(qa_result)
        if len(qa_result) > 0:
            qa_dict[video_id] = qa_result
        else:
            qa_dict[video_id] = "No Answer Available"

        if qa_ind % 10 == 0 or qa_ind == 500:
            if len(str(qa_ind)) ==1:
                checkpoint_ind = "00" + str(qa_ind)
            elif len(str(qa_ind)) == 2:
                checkpoint_ind = "0" + str(qa_ind)
            else:
                checkpoint_ind = str(qa_ind)
            checkpoint_file = f"{qa_filepath}/checkpoint_{checkpoint_ind}_{video_id}.json"

            with open(checkpoint_file, 'w', encoding='utf-8') as f:
                json.dump(qa_dict, f, ensure_ascii=False, indent=4)
        print("---")
    else:
        print("****")
        print(f'Transcript is too long: {video_id} {len(qa_query.split())}')
        print(qa_query)
        continue
    qa_ind += 1

---
---
---
---
---
---
---
---
---
---
---


In [9]:
#refactor to save at end as well
with open(out_filepath, 'w', encoding='utf-8') as f:
    json.dump(qa_dict, f, ensure_ascii=False, indent=4)